Instationary Transport Equation
===
$\DeclareMathOperator{\opdiv}{div}$

The time dependent transport equation is to find an $u = u(x,t)$ such that

$$
\frac{\partial u}{\partial t} + \opdiv bu = f
$$

with an given initial condition $u(x,t=0) = u_0(x)$, and a boundary condition on $\Gamma_{in}$.

The explicit Euler time-discretization method is to find $u^n \approx u(t_n)$:

$$
\frac{1}{t_n - t_{n-1}} \int_\Omega (u^n - u^{n-1}) v + A^{DG} (u^{n-1},v) = f(v)
$$

Here, $A^{DG}(.,.)$ is the bilinear-form of the stationary transport equation.

To compute the new coefficient vector for the new $u^n$, we have to solve an equation with the mass matrix

$$
M (u^n-u^{n-1}) = \tau \, (f - A^{DG} u^{n-1})
$$

Here, we realize the second advantage of DG methods: Since basis functions are defined element by element, the mass matrix is block diagonal. Thus, it is cheap to invert it.

In [1]:
import netgen.gui
from ngsolve import *
%gui tk
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [2]:
b = CoefficientFunction( (y-0.5, 0.5-x) )
Draw (b, mesh, "wind")

In [3]:
fes = L2(mesh, order=3)

u = fes.TrialFunction()
v = fes.TestFunction()
a = BilinearForm(fes, nonassemble=True)

a += SymbolicBFI(-b*u*grad(v))

# the upwind-term:
n = specialcf.normal(2)
uup = IfPos(b*n, u, u.Other(bnd=0))
a += SymbolicBFI(b*n*uup*v, element_boundary=True)

f = LinearForm(fes)
f.Assemble()

gfu = GridFunction(fes)
gfu.Set(exp(-10**2*((x-0.5)*(x-0.5) +(y-0.75)*(y-0.75))))

Draw(gfu, min=0, max=1, autoscale=False)

In [4]:
tau = 0.001
tend = 10
t = 0

# we need a help vector
w = gfu.vec.CreateVector()

with TaskManager():
    while t < tend:
        # apply the transport operator 
        a.Apply (gfu.vec, w)
        
        # use an efficient, matrix-free technique 
        # to solve with the mass matrix
        fes.SolveM (rho=CoefficientFunction(1), vec=w)
        gfu.vec.data -= tau * w
        t += tau
        Redraw()
    